### Data Loading and Preparation

In [1]:
import pandas as pd
import numpy as np

path = "/content/Nat_Gas.csv"
try:
    df = pd.read_csv(path, parse_dates=['Dates'], date_format="%m/%d/%y")
except Exception:
    df = pd.read_csv(path)
    df['Dates'] = pd.to_datetime(df.get('Dates', df.columns[0]), errors='coerce')

df.columns = [c.strip() for c in df.columns]
if 'Dates' in df.columns: df = df.rename(columns={'Dates':'Date'})
if 'Prices' in df.columns: df = df.rename(columns={'Prices':'Price'})
df = df.sort_values('Date').reset_index(drop=True)

print("Loaded rows:", len(df), "Date range:", df['Date'].min().date(), "to", df['Date'].max().date())

Loaded rows: 48 Date range: 2020-10-31 to 2024-09-30


### Simple Price Estimator

In [2]:
from sklearn.linear_model import LinearRegression


start = df['Date'].min()
days = (df['Date'] - start).dt.days.values.reshape(-1, 1)
trend_model = LinearRegression().fit(days, df['Price'].values)

def get_price(date):
    """Return estimated price for any date (pd.Timestamp or string)."""
    date = pd.to_datetime(date)
    end = df['Date'].max()
    if date <= end:

        daily = df.set_index('Date').reindex(pd.date_range(start, end, freq='D'))
        daily['Price'] = daily['Price'].interpolate()
        return float(daily.loc[date, 'Price'])
    else:

        d = (date - start).days
        return float(trend_model.predict([[d]])[0])

### Storage Contract Pricing

In [3]:
def price_storage_contract(injection_events, withdrawal_events,
                           max_volume=1e6,
                           storage_fee_per_month_per_1e6=100000,
                           inj_withdraw_fee_per_1e6=10000,
                           transport_fee_per_transaction=50000):
    """
    injection_events: list of (date, volume)
    withdrawal_events: list of (date, volume)
    volumes in same units as max_volume (e.g., MMBtu)
    Fees default shown can be overridden.
    Returns dict of cashflows and net value.
    """
    inj = sorted([(pd.to_datetime(d), float(v)) for d,v in injection_events], key=lambda x: x[0])
    wdr = sorted([(pd.to_datetime(d), float(v)) for d,v in withdrawal_events], key=lambda x: x[0])

    if sum(v for _,v in inj) < sum(v for _,v in wdr):
        return {"error":"Total withdrawal volume exceeds injected volume (infeasible)"}

    inventory = [[d, v, get_price(d)] for d,v in inj]

    total_cost = total_proceed = total_storage_cost = total_inj_withdraw_fees = total_transport_fees = 0.0

    for wd_date, wd_vol in wdr:
        remain = wd_vol
        sell_price = get_price(wd_date)
        while remain > 0 and inventory:
            in_date, in_rem, in_price = inventory[0]
            take = min(remain, in_rem)
            # cashflows
            total_cost += take * in_price
            total_proceed += take * sell_price
            # storage fee: pro-rata by days held (30 days ~ 1 month)
            days_held = max(0, (wd_date - in_date).days)
            months = days_held / 30.0
            total_storage_cost += storage_fee_per_month_per_1e6 * months * (take / 1e6)
            # injection/withdraw fee (per volume)
            total_inj_withdraw_fees += inj_withdraw_fee_per_1e6 * (take / 1e6)
            # transport fees: simple fixed cost counted once per matched withdrawal batch (in+out simplified)
            total_transport_fees += 2 * transport_fee_per_transaction
            # update inventory
            inventory[0][1] -= take
            if inventory[0][1] <= 1e-9:
                inventory.pop(0)
            remain -= take

    net_value = total_proceed - total_cost - total_storage_cost - total_inj_withdraw_fees - total_transport_fees

    return {
        "total_cost_buy": round(total_cost,2),
        "total_proceed_sell": round(total_proceed,2),
        "storage_costs": round(total_storage_cost,2),
        "inj_withdraw_fees": round(total_inj_withdraw_fees,2),
        "transport_fees": round(total_transport_fees,2),
        "net_value": round(net_value,2)
    }

### Example Usage

In [4]:
# show example prices
print("Example price checks:")
print("  Price 2024-06-01:", round(get_price("2024-06-01"), 2))
print("  Price 2024-12-01:", round(get_price("2024-12-01"), 2))
inj_events = [("2024-06-01", 1e6)]
wdr_events = [("2024-12-01", 1e6)]
val = price_storage_contract(inj_events, wdr_events)
print("\nContract valuation (example):")
for k,v in val.items():
    print(f"  {k}: {v}")

Example price checks:
  Price 2024-06-01: 11.4
  Price 2024-12-01: 12.2

Contract valuation (example):
  total_cost_buy: 11403333.33
  total_proceed_sell: 12204342.54
  storage_costs: 610000.0
  inj_withdraw_fees: 10000.0
  transport_fees: 100000.0
  net_value: 81009.2
